<a href="https://colab.research.google.com/github/SamuelXJames/RealRes/blob/main/models/generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.initializers import RandomUniform
from tensorflow.keras.layers import Conv2D, concatenate, Input, Add
from tensorflow.keras.models import Model


class Build_generator:

  def __init__(self):
    self.initializer = RandomUniform(minval=-0.05, maxval=0.05, seed=None)
    self.num_RDB_blocks = 6
    self.num_RDB_in_RRDB_blocks = 3
    self.num_RRDB_blocks = 23
    self.beta = 0.2
    self.scale_factor = 4
    self.path_size = None # What is the patch size

  # Residual dense block
  def RDB(self,input_layer):
    x = input_layer

    for i in range(self.num_RDB_blocks):
      layer = Conv2D(filters = 32, kernel_size = (3,3), 
                strides = (1,1), padding = 'same',
                activation = 'relu',initializer = self.initializer,
                name = 'RDB Layer {}'.format(i+1))(input_layer)
      
      x = concatenate([x,layer],axis = 3, name = 'RDB Concatenate Layer')

    # Maybe use 32 filters instead of 64 / activation ?
    x = Conv2D(filters = 32, kernel_size = (3,3),
              stides = (1,1), padding = 'same',
              initializer = self.initializer,
              name = 'Final RDB Layer')(x)

    return x

  # Residual in Residual Dense Block
  def RRDB(self,input_layer):
    
    x = input_layer
    for i in range (num_RDB_in_RRDB_blocks):
      rdb = self.RDG(x)*self.beta
      x = Add(name = 'RRDB Layer {}'.format(i+1))([x,rdb])
    
    x = x*self.beta
    x = Add(name = 'RRDB')[input_layer,x]

    return x

  def pixel_shuffle(self,input_layer):
    x = Conv2D(filters = 3*self.scale_factor**2, kernel_size = (3,3),
              strides = (1,1), padding = 'same', 
              kernel_initialize = self.initializer,
              name = 'Preshuffle')(input_layer)  
    
    return tf.nn.depth_to_space(x, block_size = self.scale_factor, data_format = 'NHWC')

  def build_generator(self):
    LR_input = Input(shape = (self.path_shape, self.path_shape,3),
                     name = 'LR_INPUT')

    
    pre_block = Conv2D(filters = 32, kernel = (3,3),
                      strides = (1,1), padding = 'same',
                      kernel_initializer = initializer,
                      name = 'Genenerator Preblock')(LR_input)
    
    
    x = self.RRDB(pre_block)
    
    for n in range(self.num_RRDB_blocks):
      x = self.RRDG(x)
    
    post_block = Conv2D(filters = 32, kernel_size = (3,3),
                        strides = (1,1), padding = 'same',
                        kernel_initializer = self.initializer,
                        name = 'Generator Post Block')(x)

    # Global Residual Learning
    GRL = Add(name = 'Global Residual Learning')([post_block,pre_block])

    # Pixel shuffeling
    PS = self.pixel_shuffle(GRL)

    # SR Image
    SR_output = Conv2D(filters = 3, kernel_size = (3,3),
                srides = (1,1), padding = 'same',
                kernel_initializer = self.initializer,
                name = 'SR Output')(PS)
    
    return Model(inputs = LR_input, outputs = SR_output)





    













  


